---

## Imports

In [1]:
import pandas as pd
import numpy as np

import pickle
import wget, os
import time
import glob
import pytz

In [2]:
# from sklearn.preprocessing import StandardScaler

import statsmodels.api as sm
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.statespace.sarimax import SARIMAX


---

## Define Functions

In [3]:
# From Matt Brems lecture

def MSE(true, predicted):
    squared_diff = np.square(true - predicted)
    return np.mean(squared_diff)

# Root Mean Square Error
def RMSE(true, predicted):
    squared_diff = np.square(true - predicted)    
    return np.sqrt(np.mean(squared_diff))

# R-squared, coefficient of determination
def R_squared(true, predicted):
    true      = np.array(true)
    predicted = np.array(predicted)
    sum_squared_diff = sum(np.square(true - predicted))
    variance  = sum(np.square(true - np.mean(true)))
    calc_r2   = 1 - (sum_squared_diff / variance)
    return calc_r2

---

## Load Pickles:  Train/Test Dataframes &  Scaled Arrays

In [4]:
with open('../data/processed/train_sc_df.pkl', 'rb') as f:
    train_sc_df = pickle.load(f)
    
with open('../data/processed/test_sc_df.pkl', 'rb') as f:
    test_sc_df = pickle.load(f)
        
with open('../data/pre_processed_df.pkl', 'rb') as f:
    df = pickle.load(f)

with open('../data/processed/train.pkl', 'rb') as f:
    train = pickle.load(f)
        
with open('../data/processed/test.pkl', 'rb') as f:
    test = pickle.load(f)


In [5]:
exog_col_for_dam = [train.columns[i] for i, col in enumerate(df.columns) if col != 'dam_price_per_mwh']
exog_col_for_hasp = [train.columns[i] for i, col in enumerate(df.columns) if col != 'hasp_price_per_mwh']

In [6]:
X_train_sc_dam = train_sc_df[exog_col_for_dam]
X_test_sc_dam  = test_sc_df[exog_col_for_dam]
y_train_dam    = train['dam_price_per_mwh']
y_test_dam     = test['dam_price_per_mwh']

In [7]:
X_train_sc_hasp = train_sc_df[exog_col_for_hasp]
X_test_sc_hasp  = test_sc_df[exog_col_for_hasp]
y_train_hasp    = train['hasp_price_per_mwh']
y_test_hasp     = test['hasp_price_per_mwh']

In [8]:
y_train_dam.shape

(21800,)

In [9]:
X_train_sc_dam.shape

(21800, 27)

---

## Gridsearch to find best MSE params for `P`, `D`, `Q`, and `S`

(for `DAM` and `HASP` prices as endogenous variables,  
 use the ARIMA parameters for `p`, `d`, & `q` found in previous ARIMA gridsearch...  
 `DAM: 4, 0, 6`  and  `HASP: 6, 0 ,6`)

> NOTE: Because of the amount of time each SARIMAX model takes to fit, there was simply not enough time to do these grid searches, although the code is correct and ready to run

In [ ]:
# Matt Brem's grid search loop
# Starting MSE and (P, D, Q).

# mse = 99 * (10 ** 16)
# final_P = 0
# final_D = 0
# final_Q = 0
# final_S = 0

# for P in range(3):
#     for Q in range(3):
#         for D in range(3):
#             for S in range(0,24,8):
#                 try:
#                     # Instantiate SARIMA model.
#                     sarima = SARIMAX(endog = y_train_dam,
#                                      order = (4, 0, 6),              # (p, d, q)
#                                      seasonal_order = (P, D, Q, S),  # (P, D, Q, S)
#                                      exog = X_train_sc_dam) 

#                     # Fit SARIMA model.
#                     model = sarima.fit()

#                     # Generate predictions based on training set.
#                     preds = model.predict()

#                     # Evaluate predictions.
#                     print(f'MSE for (4,0,6) x ({P},{D},{Q},{S}) ... {mean_squared_error(train["dam_price_per_mwh"], preds)}')

#                     # Save for final report.
#                     if mse > mean_squared_error(train['dam_price_per_mwh'], preds):
#                         mse = mean_squared_error(train['dam_price_per_mwh'], preds)
#                         final_P = P
#                         final_D = D
#                         final_Q = Q
#                         final_S = S

#                 except:
#                     pass

# print(f'Our model that minimizes MSE on the training data is the SARIMA(4,0,6) x ({final_P},{final_D},{final_Q},{final_S})')
# print(f'This model MSE = {mse}')

In [ ]:
# Matt Brem's grid search loop
# Starting MSE and (P, D, Q).

# mse = 99 * (10 ** 16)
# final_P = 0
# final_D = 0
# final_Q = 0
# final_S = 0

# for P in range(2):
#     for Q in range(2):
#         for D in range(2):
#             for S in range(0,25,12):
#                 try:
#                     # Instantiate SARIMA model.
#                     sarima = SARIMAX(endog = y_train_hasp],
#                                      order = (6, 0, 6),              # (p, d, q)
#                                      seasonal_order = (P, D, Q, S),  # (P, D, Q, S)
#                                      exog = X_train_sc_hasp) 

#                     # Fit SARIMA model.
#                     model = sarima.fit()

#                     # Generate predictions based on training set.
#                     preds = model.predict()

#                     # Evaluate predictions.
#                     print(f'MSE for (6,0,6) x ({P},{D},{Q},{S}) ... {mean_squared_error(train["hasp_price_per_mwh"], preds)}')

#                     # Save for final report.
#                     if mse > mean_squared_error(train['hasp_price_per_mwh'], preds):
#                         mse = mean_squared_error(train['hasp_price_per_mwh'], preds)
#                         final_P = P
#                         final_D = D
#                         final_Q = Q
#                         final_S = S

#                 except:
#                     pass

# print(f'Our model that minimizes MSE on the training data is the SARIMA(6,0,6) x ({final_P},{final_D},{final_Q},{final_S})')
# print(f'This model MSE = {mse}')

## Instantiate and fit the models with best params

In [10]:
P = 0
D = 1
Q = 0
S = 24

# Instantiate SARIMA model.
dam_sarimax01024 = SARIMAX(endog = y_train_dam,
                 order = (4, 0, 6),              # (p, d, q)
                 seasonal_order = (P, D, Q, S),  # (P, D, Q, S)
                 exog = X_train_sc_dam) 

# Fit SARIMA model.
dam_sarimax01024_model = dam_sarimax01024.fit()

# Generate predictions based on training set.
dam_sarimax01024_preds = dam_sarimax01024_model.predict()

# Evaluate predictions.
print(f'MSE for (4,0,6) x ({P},{D},{Q},{S}) ... {mean_squared_error(train["dam_price_per_mwh"], dam_sarimax01024_preds):.2f}')
 


/Users/Mehrdadi/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
/Users/Mehrdadi/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           38     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.75936D+00    |proj g|=  1.35750D-02


 This problem is unconstrained.



At iterate    5    f=  3.74634D+00    |proj g|=  4.73161D-03

At iterate   10    f=  3.74449D+00    |proj g|=  1.22084D-02

At iterate   15    f=  3.74430D+00    |proj g|=  2.84213D-03

At iterate   20    f=  3.74422D+00    |proj g|=  7.80804D-04

At iterate   25    f=  3.74412D+00    |proj g|=  4.95879D-03

At iterate   30    f=  3.74405D+00    |proj g|=  1.65648D-03

At iterate   35    f=  3.74400D+00    |proj g|=  3.07978D-03

At iterate   40    f=  3.74392D+00    |proj g|=  1.82710D-03

At iterate   45    f=  3.74389D+00    |proj g|=  5.15260D-04

At iterate   50    f=  3.74386D+00    |proj g|=  3.76572D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tn

/Users/Mehrdadi/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


MSE for (4,0,6) x (0,1,0,24) ... 105.47


In [11]:
with open('../fitted_models/dam_sarimax01024_model.pkl', 'wb') as f:
    pickle.dump(dam_sarimax01024_model, f)
    
with open('../data/predictions/dam_sarimax01024_preds.pkl', 'wb') as f:
    pickle.dump(dam_sarimax01024_preds, f)

In [12]:
P = 0
D = 1
Q = 0
S = 24

# Instantiate SARIMA model.
hasp_sarimax01024 = SARIMAX(endog = y_train_hasp,
                 order = (6, 0, 6),              # (p, d, q)
                 seasonal_order = (P, D, Q, S),  # (P, D, Q, S)
                 exog = X_train_sc_hasp) 

# Fit SARIMA model.
hasp_sarimax01024_model = hasp_sarimax01024.fit()

# Generate predictions based on training set.
hasp_sarimax01024_preds = hasp_sarimax01024_model.predict()

# Evaluate predictions.
print(f'MSE for (6,0,6) x ({P},{D},{Q},{S}) ... {mean_squared_error(train["hasp_price_per_mwh"], hasp_sarimax01024_preds):.2f}')
 


/Users/Mehrdadi/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
/Users/Mehrdadi/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           40     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  5.10253D+00    |proj g|=  7.60475D-02

At iterate    5    f=  5.09971D+00    |proj g|=  1.84246D-02

At iterate   10    f=  5.09389D+00    |proj g|=  6.07601D-02

At iterate   15    f=  5.08616D+00    |proj g|=  6.14065D-02

At iterate   20    f=  5.08357D+00    |proj g|=  1.82236D-02

At iterate   25    f=  5.07888D+00    |proj g|=  1.13175D-01

At iterate   30    f=  5.07382D+00    |proj g|=  2.92868D-01

At iterate   35    f=  5.07270D+00    |proj g|=  1.65504D-01

At iterate   40    f=  5.07177D+00    |proj g|=  3.44070D-01

At iterate   45    f=  5.06977D+00    |proj g|=  1.55898D-01


/Users/Mehrdadi/opt/anaconda3/lib/python3.8/site-packages/statsmodels/base/model.py:566: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "



At iterate   50    f=  5.06769D+00    |proj g|=  1.92445D-01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
   40     50     63      1     0     0   1.924D-01   5.068D+00
  F =   5.0676899965518665     

STOP: TOTAL NO. of ITERATIONS REACHED LIMIT                 
MSE for (6,0,6) x (0,1,0,24) ... 1490.58


In [13]:
with open('../fitted_models/hasp_sarimax01024_model.pkl', 'wb') as f:
    pickle.dump(hasp_sarimax01024_model, f)
    
with open('../data/predictions/hasp_sarimax01024_preds.pkl', 'wb') as f:
    pickle.dump(hasp_sarimax01024_preds, f)